## Get a filtration from Gudhi

In [1]:
import gudhi as gd
import tadasets
sphere = tadasets.dsphere(n=30, d=1, r=1, noise=0, seed=42)
alpha = gd.AlphaComplex(points=sphere)
st = alpha.create_simplex_tree()
filtration = list(st.get_filtration())

## Convert it to boundary map data

In [2]:
from persistent_laplacians.filtrations import simplices_by_dimension, compute_boundary_matrices, filtration_hash_map

unique_filtration_values = sorted(list(set([f for (_, f) in filtration])))
simplices_by_dim, simplices_by_dim_only_filt = simplices_by_dimension(filtration)
boundary_matrices = compute_boundary_matrices(simplices_by_dim)
boundary_maps_index_dict = filtration_hash_map(filtration, simplices_by_dim_only_filt)

In [3]:
boundary_matrices

{1: {'n_rows': 30,
  'n_cols': 57,
  'data': array([ 1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1.,
         -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,
          1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1.,
         -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,
          1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1.,
         -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,
          1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1.,
         -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,
          1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.]),
  'rows': array([22, 14,  5,  2, 13,  8, 18,  6, 28,  7, 19, 12, 18, 11, 16, 11, 24,
          6, 29,  1, 23, 15, 28, 14, 12,  8, 25, 17, 10,  0, 26,  4, 21,  9,
          9,  5, 20,  3, 27,  3, 23,  4, 22,  2, 21, 16, 20, 10, 24, 17, 26,
         13, 27,  1, 19,  7, 25,  0, 29, 15, 14,  2, 16,  6, 13, 

## Compute homology using persistent laplacians
For now, compute only homology (using SVD)

In [4]:
from persistent_laplacians import persistent_laplacians as pl
result = pl.process_tda(
    boundary_matrices,
    boundary_maps_index_dict
)

## Barcodes 

In [5]:
from persistent_laplacians.barcodes import compute_barcodes
print({k: v for k,v in result[1].items() if v > 0})
barcodes = compute_barcodes(result, unique_filtration_values)
print(barcodes)
barcodes = [
    (q, (unique_filtration_values[i], unique_filtration_values[j]))
    for q in barcodes.keys()
    for (i, j) in barcodes[q].keys()
]
sorted(barcodes)

{(47, 49): 4, (52, 56): 1, (30, 34): 1, (31, 51): 1, (51, 51): 5, (33, 54): 1, (42, 44): 1, (38, 45): 1, (37, 52): 1, (39, 40): 1, (30, 41): 1, (40, 53): 1, (33, 49): 1, (31, 38): 1, (42, 51): 1, (45, 45): 3, (49, 51): 5, (41, 54): 1, (35, 49): 1, (41, 45): 1, (39, 49): 1, (41, 53): 1, (34, 51): 1, (42, 52): 1, (41, 43): 1, (46, 55): 1, (34, 34): 2, (37, 43): 1, (36, 43): 1, (40, 50): 1, (40, 42): 1, (41, 51): 1, (47, 47): 4, (33, 33): 2, (30, 47): 1, (44, 53): 2, (34, 41): 1, (48, 49): 4, (43, 45): 1, (40, 45): 1, (39, 56): 1, (37, 49): 1, (38, 43): 1, (30, 37): 1, (32, 33): 2, (38, 46): 1, (31, 49): 1, (33, 53): 1, (30, 56): 1, (37, 45): 1, (41, 56): 1, (43, 44): 1, (48, 53): 3, (51, 53): 3, (41, 55): 1, (44, 44): 2, (39, 48): 1, (53, 54): 2, (42, 56): 1, (37, 56): 1, (34, 52): 1, (34, 39): 1, (35, 56): 1, (38, 38): 2, (33, 46): 1, (39, 42): 1, (34, 46): 1, (31, 33): 1, (45, 50): 3, (44, 51): 2, (34, 44): 1, (41, 41): 1, (32, 43): 1, (32, 51): 1, (30, 42): 1, (47, 55): 1, (38, 51): 1

[(1, (0.09938374319148695, 1.0000000000000697)),
 (1, (0.9999999999993767, 0.9999999999996827)),
 (1, (0.9999999999998734, 0.9999999999999882)),
 (1, (0.9999999999999973, 1.0000000000000027)),
 (1, (0.9999999999999978, 1.0000000000000016)),
 (1, (0.9999999999999991, 1.0000000000000007)),
 (1, (0.9999999999999998, 0.9999999999999999)),
 (1, (0.9999999999999998, 1.0000000000000004))]

## Verify with Gudhi

In [6]:
gudhi_persistence = [(q, (i, j)) for (q, (i,j)) in st.persistence() if q > 0]
print(sorted(gudhi_persistence))
sorted(gudhi_persistence) == sorted(barcodes)

[(1, (0.09938374319148695, 1.0000000000000697)), (1, (0.9999999999993767, 0.9999999999996827)), (1, (0.9999999999998734, 0.9999999999999882)), (1, (0.9999999999999973, 1.0000000000000027)), (1, (0.9999999999999978, 1.0000000000000016)), (1, (0.9999999999999991, 1.0000000000000007)), (1, (0.9999999999999998, 0.9999999999999999)), (1, (0.9999999999999998, 1.0000000000000004))]


True